# 🎤 Clone Voice với RVC (Retrieval-based Voice Conversion)
Notebook để train và sử dụng RVC clone giọng nói trên Google Colab

**Yêu cầu:** GPU T4/A100 (chọn Runtime > Change runtime type > GPU)

In [ ]:
#@title 1️⃣ KEEP ALIVE (Chạy ngầm tránh disconnect)
import time, random, threading, gc
from IPython.display import display, Javascript

# JavaScript Keep Alive
display(Javascript('''
(function(){
    let count = 0;
    function keepAlive() {
        count++;
        console.log("Keep alive tick: " + count);
        document.querySelector("colab-connect-button")?.click();
        
        // Sim UI update
        var status = document.getElementById("ka-status");
        if (status) status.innerHTML = "🔄 Active: " + count;

        setTimeout(keepAlive, 25000 + Math.random() * 15000);
    }
    
    // Create Status UI
    var div = document.createElement("div");
    div.id = "ka-status";
    div.innerHTML = "🔄 ON";
    div.style.cssText = "position: fixed; top: 10px; right: 10px; background: linear-gradient(135deg, #FF6B6B, #556270); color: white; padding: 8px 15px; border-radius: 20px; z-index: 9999; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.1);";
    document.body.appendChild(div);
    
    keepAlive();
})();
'''))

# Python Keep Alive Background Thread
def py_keep_alive():
    while True:
        time.sleep(random.randint(30, 90))
        gc.collect() # Memory cleanup to fake activity
        
threading.Thread(target=py_keep_alive, daemon=True).start()
print("✅ Keep Alive activated!")

In [ ]:
#@title 2️⃣ Kiểm tra GPU
!nvidia-smi

In [ ]:
#@title 3️⃣ Clone RVC và cài đặt dependencies
%cd /content

# Clone RVC WebUI
!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
%cd Retrieval-based-Voice-Conversion-WebUI

# Cài dependencies
!pip install -q -r requirements.txt
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q faiss-gpu
!pip install -q gradio==3.50.2
!pip install -q edge-tts  # Thư viện TTS để tạo giọng đọc mẫu

print("✅ Đã cài đặt xong!")

In [ ]:
#@title 4️⃣ Download pretrained models
%cd /content/Retrieval-based-Voice-Conversion-WebUI

# Download Hubert model
!mkdir -p assets/hubert
!wget -q -O assets/hubert/hubert_base.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt

# Download RMVPE model (pitch extraction)
!mkdir -p assets/rmvpe
!wget -q -O assets/rmvpe/rmvpe.pt https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt

# Download pretrained models
!mkdir -p assets/pretrained_v2
!wget -q -O assets/pretrained_v2/f0G48k.pth https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G48k.pth
!wget -q -O assets/pretrained_v2/f0D48k.pth https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D48k.pth

print("✅ Đã download xong pretrained models!")

In [ ]:
#@title 5️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục lưu trữ
!mkdir -p /content/drive/MyDrive/rvc_training/audio
!mkdir -p /content/drive/MyDrive/rvc_training/models
print("✅ Đã mount Drive và tạo thư mục!")

In [ ]:
#@title 6️⃣ Cấu hình Training

# === CẤU HÌNH CHÍNH ===
MODEL_NAME = "my_voice"  #@param {type:"string"}

# Thư mục chứa file audio training (upload vào Drive)
AUDIO_DIR = "/content/drive/MyDrive/rvc_training/audio"  #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/rvc_training/models"  #@param {type:"string"}

# Training parameters
SAMPLE_RATE = 48000  #@param [40000, 48000]
BATCH_SIZE = 8  #@param {type:"integer"}
TOTAL_EPOCHS = 100  #@param {type:"integer"}
SAVE_EVERY_EPOCH = 10  #@param {type:"integer"}

# F0 method
F0_METHOD = "rmvpe"  #@param ["rmvpe", "crepe", "harvest"]

print(f"✅ Cấu hình xong! Model: {MODEL_NAME}")

In [ ]:
#@title 7️⃣ Tiền xử lý Audio (Train Only)
%cd /content/Retrieval-based-Voice-Conversion-WebUI

import os
import shutil

# Copy audio files vào thư mục training
train_dir = f"logs/{MODEL_NAME}"
os.makedirs(train_dir, exist_ok=True)

# Kiểm tra audio files
if os.path.exists(AUDIO_DIR):
    audio_files = [f for f in os.listdir(AUDIO_DIR) if f.endswith(('.wav', '.mp3', '.flac', '.ogg'))]
    print(f"Tìm thấy {len(audio_files)} file audio:")
    for f in audio_files:
        print(f"  - {f}")
    
    if len(audio_files) > 0:
        !python infer/modules/train/preprocess.py "{AUDIO_DIR}" {SAMPLE_RATE} 2 "logs/{MODEL_NAME}" False 3.0
        print("\n✅ Đã tiền xử lý audio xong!")
    else:
        print("⚠️ Không tìm thấy file audio để train. (Nếu chỉ dùng Inference thì bỏ qua)")
else:
    print("⚠️ Thư mục audio chưa tồn tại. (Nếu chỉ dùng Inference thì bỏ qua)")

In [ ]:
#@title 8️⃣ Extract Features & Train (Train Only)
# SKIP nếu bạn chỉ chạy Inference
%cd /content/Retrieval-based-Voice-Conversion-WebUI

# Extract features
import os
if os.path.exists(f"logs/{MODEL_NAME}") and os.listdir(f"logs/{MODEL_NAME}"):
    !python infer/modules/train/extract/extract_feature_print.py cuda:0 1 0 0 "logs/{MODEL_NAME}" "v2"
    !python infer/modules/train/extract_f0_print.py "logs/{MODEL_NAME}" 2 "{F0_METHOD}"

    # Train
    !python infer/modules/train/train.py -e "{MODEL_NAME}" -sr {SAMPLE_RATE} -f0 1 -bs {BATCH_SIZE} -te {TOTAL_EPOCHS} -se {SAVE_EVERY_EPOCH} -pg "assets/pretrained_v2/f0G48k.pth" -pd "assets/pretrained_v2/f0D48k.pth" -l 0 -c 0 -sw 0 -v "v2"

    # Create Index
    !python infer/modules/train/extract_index.py "logs/{MODEL_NAME}" "v2"
    
    print("✅ Training hoàn tất!")
else:
    print("⚠️ Không có dữ liệu để train.")

In [ ]:
#@title 9️⃣ Setup API Interface (TTS + RVC)
# Cài Cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import sys, os, glob, asyncio, edge_tts, subprocess, threading, time
import gradio as gr
import soundfile as sf
from google.colab import userdata

sys.path.append('/content/Retrieval-based-Voice-Conversion-WebUI')
from infer.modules.vc.modules import VC
from configs.config import Config

# Init RVC
config = Config()
vc = VC(config)

# Load Model tự động (Lấy model vừa train hoặc upload)
try:
    # Ưu tiên lấy từ Logs (model vừa train)
    model_path = glob.glob(f"logs/{MODEL_NAME}/*G_*.pth")[-1]
    # Hoặc lấy từ Weights
    # model_path = glob.glob(f"assets/weights/{MODEL_NAME}*.pth")[0]
    
    index_path = glob.glob(f"logs/{MODEL_NAME}/*.index")[-1]
    vc.get_vc(model_path)
    print(f"✅ Đã load model: {model_path}")
except Exception as e:
    print(f"⚠️ Chưa load được model: {e}. Vui lòng kiểm tra lại path.")

# --- HÀM XỬ LÝ CHÍNH ---
async def text_to_speech_edge(text, voice):
    # List voices: vi-VN-HoaiMyNeural, vi-VN-NamMinhNeural, en-US-AriaNeural...
    communicate = edge_tts.Communicate(text, voice)
    temp_file = "/content/tts_temp.mp3"
    await communicate.save(temp_file)
    return temp_file

def process_pipeline(text, voice_name, pitch_change, index_rate):
    if not text:
        return None, "Vui lòng nhập text!"
    
    # 1. TTS
    tts_start = time.time()
    try:
        audio_path = asyncio.run(text_to_speech_edge(text, voice_name))
    except Exception as e:
        return None, f"Lỗi TTS: {str(e)}"
    
    # 2. RVC Conversion
    try:
        wav_opt = vc.vc_single(
            sid=0,
            input_audio_path=audio_path,
            f0_up_key=pitch_change,
            f0_method="rmvpe",
            index_file=index_path,
            index_rate=index_rate,
            filter_radius=3,
            resample_sr=0,
            rms_mix_rate=0.25,
            protect=0.33
        )
        if wav_opt[0] == "Success":
            return (wav_opt[1][0], wav_opt[1][1]), "✅ Thành công!"
        else:
            return None, f"Lỗi RVC: {wav_opt[0]}"
    except Exception as e:
        return None, f"Lỗi Process: {str(e)}"

# --- GRADIO INTERFACE ---
with gr.Blocks(title="RVC Voice Clone API") as app:
    gr.Markdown("# 🎙️ Voice Clone API (TTS + RVC)")
    with gr.Row():
        with gr.Column():
            txt_input = gr.Textbox(label="Nhập văn bản", lines=3, placeholder="Xin chào, tôi là AI...")
            voice_sel = gr.Dropdown(label="Giọng đọc mẫu (TTS)", choices=["vi-VN-HoaiMyNeural", "vi-VN-NamMinhNeural", "en-US-AriaNeural"], value="vi-VN-HoaiMyNeural")
            pitch = gr.Slider(label="Đổi tông (Pitch)", minimum=-12, maximum=12, value=0, step=1)
            index_rate = gr.Slider(label="Index Rate (Độ giống)", minimum=0, maximum=1, value=0.75)
            btn = gr.Button("🚀 Tạo giọng nói", variant="primary")
        with gr.Column():
            audio_out = gr.Audio(label="Kết quả")
            status = gr.Textbox(label="Trạng thái")

    btn.click(process_pipeline, inputs=[txt_input, voice_sel, pitch, index_rate], outputs=[audio_out, status])

# --- CLOUDFLARE TUNNEL ---
try:
    TOKEN = userdata.get('CLOUDFLARE_TOKEN')
except:
    TOKEN = ""

if not TOKEN:
    print("⚠️ Không tìm thấy Secret 'CLOUDFLARE_TOKEN'. Chạy local mode.")
    app.queue().launch(share=True)
else:
    print("🚀 Khởi động Cloudflare Tunnel...")
    threading.Thread(target=lambda: subprocess.run(["cloudflared", "tunnel", "run", "--token", TOKEN]), daemon=True).start()
    time.sleep(5)
    print(f"🌐 API sẵn sàng tại: https://comfyui.maymoc.shop")
    app.queue().launch(server_name="0.0.0.0", server_port=8188, quiet=True)